# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aasiaat,68.7098,-52.8699,-7.85,85,93,12.03,GL,1708727807
1,1,yellowknife,62.4560,-114.3525,12.42,96,100,18.41,CA,1708727565
2,2,ust'-ilimsk,58.0006,102.6619,-9.92,98,98,5.79,RU,1708727711
3,3,port lincoln,-34.7333,135.8667,64.69,60,44,15.41,AU,1708727808
4,4,mangrol,21.1167,70.1167,67.33,38,0,10.67,IN,1708727809


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
city_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_cities_df = city_data_df.copy().loc[(city_data_df['Max Temp'] < 38) &
                                               (city_data_df['Max Temp'] > 25) &
                                               (city_data_df['Wind Speed'] > 5) &
                                               (city_data_df['Wind Speed'] < 12) &
                                               (city_data_df['Humidity'] >= 70) &
                                               (city_data_df['Humidity'] <= 90) &
                                               (city_data_df['Cloudiness'] >= 75)  
                                              ]
# Drop any rows with null values
ideal_weather_cities_df.dropna(inplace = True)

# Display sample data
ideal_weather_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,brookings,44.3114,-96.7984,33.80,81,100,11.50,US,1708727811
27,27,isafjordur,66.0755,-23.1240,25.86,87,99,10.20,IS,1708727815
94,94,longyearbyen,78.2186,15.6401,26.44,74,75,5.75,SJ,1708727747
215,215,vadso,70.0744,29.7487,35.76,75,75,11.50,NO,1708727899
244,244,xiaoweizhai,26.1903,107.5125,28.58,90,100,5.06,CN,1708727906
305,305,shuangjiang,26.1547,109.7138,27.28,88,100,5.68,CN,1708727925
353,353,egilsstadir,65.2653,-14.3948,31.91,86,75,9.22,IS,1708727940
411,411,tromso,69.6496,18.9570,37.42,70,75,11.50,NO,1708727963
423,423,guilin,25.2819,110.2864,32.11,85,100,7.20,CN,1708727967
503,503,shinjo,38.7586,140.3008,31.08,78,80,6.58,JP,1708727994


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, 'Hotel Name', '')

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,brookings,US,44.3114,-96.7984,81,
27,isafjordur,IS,66.0755,-23.1240,87,
94,longyearbyen,SJ,78.2186,15.6401,74,
215,vadso,NO,70.0744,29.7487,75,
244,xiaoweizhai,CN,26.1903,107.5125,90,
305,shuangjiang,CN,26.1547,109.7138,88,
353,egilsstadir,IS,65.2653,-14.3948,86,
411,tromso,NO,69.6496,18.9570,70,
423,guilin,CN,25.2819,110.2864,85,
503,shinjo,JP,38.7586,140.3008,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories': 'accommodation.hotel',
          'apiKey':geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = hotel_df.loc[index, 'Lat']
    Longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
brookings - nearest hotel: Quality Inn
isafjordur - nearest hotel: Hótel Horn
longyearbyen - nearest hotel: The Vault
vadso - nearest hotel: Scandic Vadsø
xiaoweizhai - nearest hotel: No hotel found
shuangjiang - nearest hotel: No hotel found
egilsstadir - nearest hotel: Valaskjálf
tromso - nearest hotel: Clarion Collection Hotel Aurora
guilin - nearest hotel: 丰源酒店
shinjo - nearest hotel: 食彩の宿おくやま
quesnel - nearest hotel: Quality Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
12,brookings,US,44.3114,-96.7984,81,Quality Inn
27,isafjordur,IS,66.0755,-23.1240,87,Hótel Horn
94,longyearbyen,SJ,78.2186,15.6401,74,The Vault
215,vadso,NO,70.0744,29.7487,75,Scandic Vadsø
244,xiaoweizhai,CN,26.1903,107.5125,90,No hotel found
305,shuangjiang,CN,26.1547,109.7138,88,No hotel found
353,egilsstadir,IS,65.2653,-14.3948,86,Valaskjálf
411,tromso,NO,69.6496,18.9570,70,Clarion Collection Hotel Aurora
423,guilin,CN,25.2819,110.2864,85,丰源酒店
503,shinjo,JP,38.7586,140.3008,78,食彩の宿おくやま


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng', 'Lat', geo = True, color = 'City', alpha = 0.8, size = 'Humidity', tiles = 'OSM', hover_cols = ['Hotel Name', 'Country'])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)